<a href="https://colab.research.google.com/github/Junperr/TIPE/blob/main/AI/single_Station.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importation


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
from IPython.display import clear_output
from six.moves import urllib
import sqlite3 as sql
import tensorflow.compat.v2.feature_column as fc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import shapely.geometry as geo
from scipy.interpolate import griddata
import numpy as np
from scipy import ndimage
import tensorflow as tf
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Info/Tipe/sismic1.db

# Parcours des données

ci dessous l'on définit nos set de donnée la partie pour entrainer le model et celle pour vérifier que l'on peut généraliser notre model (on verifie sa précision sur des données qu'il n'a jamais vus)

In [ ]:
con=sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db')
table_earth = pd.read_sql("""SELECT * FROM seismes order by mag desc""",con)
table_sta = pd.read_sql("""SELECT * FROM station """,con)
table_info = pd.read_sql("""SELECT * FROM infos """,con)
print('infos :',table_info.head(),'earthquake :',table_earth.head(),"station :",table_sta.head(),sep='\n'+'####'+'\n')
# station_list = pd.read_sql("""SELECT PGA,PGV,mean,std,PGA_UD,PGA_EW,PGA_NS FROM infos """,sql.connect('sismic1.db'))

dftrain = pd.read_sql("""
SELECT st.code,earthquake_id,mag,PGA,PGV,mean,std,PGA_UD,PGA_EW,PGA_NS FROM infos join station as st
on st.code = infos.station_code join seismes as se
on se.id==infos.earthquake_id
Group By earthquake_id
Order by mag DESC """,sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db'))
print('dftrain',dftrain.head(),sep='\n')
# print(dftrain,dftrain.loc[0], sep='\n')
print(len(dftrain))

Quelque graphes pour visualiser la distributions des données:

In [ ]:
dfstat_seisme = pd.read_sql("""
SELECT AVG(nbr_record) as nbr_moy_record,count(*) as nbr_earth,mag from (
SELECT count(*) as nbr_record,se.id,se.mag FROM infos join  seismes as se
on se.id==infos.earthquake_id
group by se.id
having nbr_record >20
)
group by mag
order by mag desc""",sql.connect('/content/drive/MyDrive/Info/Tipe/sismic1.db'))
#dftrain['nbr_record']
ax = dfstat_seisme.plot.scatter(x='mag', y='nbr_moy_record', c='nbr_earth',grid=True,use_index=True,figsize=(15,7),xlabel="magnitude",cmap='winter',colorbar=True)
plt.ylabel('nombre moyen de station',fontsize=20)
plt.xlabel('magnitude',fontsize=20)
plt.title(label = 'distribution des séismes ',fontsize=20)
ax.set_xticks([0.5*x for x in range(4,20)])
ax.set_xticklabels([0.5*x for x in range(4,17)])
plt.show()

In [ ]:
dftrain.mag.hist(bins=50).set_xlabel("magnitude")

# Data Normalisation and selection


In [ ]:
order_sta = pd.read_sql("""SELECT Count(*) as nb_seismes,station_code FROM infos
                            Group By station_code
                            Order By nb_seismes DESC""",con)
station_model = order_sta['station_code'][0] # pour l'instant on effectue un model sur la station ayant enregistré le plus de seismes
data_set = pd.read_sql("""  SELECT PGA,mag,long,lat,depth FROM infos AS i Join seismes AS s
                            On i.earthquake_id = s.id
                            Where i.station_code = '{}'
                             """.format(str(station_model)),con)
data_set_normed= data_set.copy()
for x in data_set.keys():
    if x == 'PGA':
        way_back = [data_set_normed[x].mean(),data_set_normed[x].std()]
    data_set_normed[x] = (data_set[x]-data_set[x].mean())/data_set[x].std()
print(data_set,data_set_normed,sep='\n')

data_set_normed = data_set_normed.sample(frac=1).reset_index(drop=True)
# shuffle our data_set, drop=True mean we do not keep old index on a new column
n=len(data_set_normed)
data_train = data_set_normed[:int(0.90*n)]
data_eval = data_set_normed[int(0.90*n):]
data_eval = data_eval.reset_index(drop=True)
result_train = data_train.pop('PGA')
result_eval = data_eval.pop('PGA')

# Model

On créer notre model:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(40000, activation=tf.nn.selu,input_shape=[4]),
  tf.keras.layers.Dense(5000, activation=tf.nn.selu),
  tf.keras.layers.Dense(units=1, input_shape=[1])
])

On le compile:


In [ ]:
model.compile(loss="mse",
              optimizer=tf.keras.optimizers.Adam(0.00001),metrics=['mae','mse'])
list_model = ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl']

In [ ]:
model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)

history = model.fit(data_train, result_train, epochs=10000,validation_split=0.2, verbose=False, callbacks=[early_stop])

In [ ]:
hist = pd.DataFrame(history.history)
for x in hist.keys():
    hist[x] = hist[x] * way_back[1]
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot([history.history['loss'][i] * way_back[1] for i in range (len(history.history['loss']))], label='loss')
  plt.plot([history.history['val_loss'][i] * way_back[1] for i in range (len(history.history['val_loss']))], label='val_loss')
  #plt.ylim([0, 100])
  #plt.xlim([0,50])
  plt.xlabel('Epoch')
  plt.ylabel('Error [PGA]')
  plt.legend()
  plt.grid(True)
plot_loss(history)

In [ ]:
# print(data_eval,result_eval,sep='\n')
X = [x for x in range (0,len(data_eval))]
# for x in X :
#   print(data_eval[x:x+1],result_eval[x:x+1])
model_predict_single = np.array([model.predict(data_eval[x:x+1]) for x in X])

In [ ]:
fig,ax = plt.subplots(figsize=(20,15))
print(way_back)
for x in X :
    ax.plot(x,(result_eval[x] * way_back[1]) + way_back[0],"b8",markersize=8)
    ax.plot(x, max((model_predict_single[x][0][0]*way_back[1]) + way_back[0],0) ,"r*",markersize=10)
for i in range (len(X)//2+len(X)%2):
    plt.axvline(2*i, ls='-.', color= 'gray')
plt.ylabel("PGA",fontsize=40)
plt.legend(["real values",'predicted values'],fontsize=25,markerscale=4)

# Stop there

In [ ]:
X1 = [x for x in range (0,len(data_train))]
model_predict_single_train = np.array([model.predict(data_train[x:x+1]) for x in X1])
fig1,ax1 = plt.subplots(figsize=(20,15))
for x in X1 :
  ax1.plot(x,(model_predict_single_train[x][0]* way_back[1]) + way_back[0] ,"k*",markersize=10)
  ax1.plot(x,(result_train[x]* way_back[1])+way_back[0],"b8",markersize=10)
plt.grid()
plt.ylabel("PGA",fontsize=40)
plt.legend(['predicted values',"real values"],fontsize=25,markerscale=4)